In [ ]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np

## Lecture 18 ##

# Randomized Control Experiment

In [ ]:
botox = Table.read_table('bta.csv')
botox.show()

In [ ]:
botox.pivot('Result', 'Group')

In [ ]:
botox.group('Group', np.average)

# Testing the Hypothesis

In [ ]:
average_array = botox.group('Group', np.average).column("Result average")
observed_diff = abs(average_array.item(0) - average_array.item(1))
observed_diff

In [ ]:
def compute_statistic(tbl, group_label, number_label):
    average_array = tbl.group(group_label, np.average).column(number_label + " average")
    return abs(average_array.item(0) - average_array.item(1))

In [ ]:
compute_statistic(botox, "Group", "Result")

In [ ]:
shuffled_labels = botox.sample(with_replacement = False).column("Group")
botox_shuffled = botox.with_column("Group", shuffled_labels)
botox_shuffled

In [ ]:
def one_simulated_difference(tbl, group_label, number_label):
    shuffled_labels = tbl.sample(with_replacement = False).column(group_label)
    botox_shuffled = botox.with_column(group_label, shuffled_labels)
    return compute_statistic(botox_shuffled, group_label, number_label)

In [ ]:
simulated_diffs = make_array()

for i in np.arange(10000):
    sim_diff = one_simulated_difference(botox, 'Group', 'Result')
    simulated_diffs = np.append(simulated_diffs, sim_diff)

In [ ]:
col_name = 'Distances between groups'
Table().with_column(col_name, simulated_diffs).hist(col_name)

In [ ]:
# p-value
sum(simulated_diffs >= observed_diff)/len(simulated_diffs)

## Example: Benford's Law

In [ ]:
digits = np.arange(1, 10)
benford_model = np.log10(1 + 1/digits)

In [ ]:
benford = Table().with_columns(
    'First digit', digits,
    'Benford model prob', benford_model)
benford.barh('First digit')

In [ ]:
# You don't have to understand how this function works, since it uses Python features from beyond Math 121.
def first_digit(num):
    return int(str(num)[0])

In [ ]:
first_digit(32)

In [ ]:
first_digit(17719087)

In [ ]:
# County populations from the census data
counties = Table.read_table('counties.csv')
counties = counties.where('SUMLEV', 50).select(5,6,9).relabeled(0,'State').relabeled(1,'County').relabeled(2,'Population')
counties.show(3)

In [ ]:
first_digits = counties.apply(first_digit, 'Population')
counties = counties.with_column('First digit', first_digits)
counties.show(3)

In [ ]:
num_counties = counties.num_rows

In [ ]:
by_digit = counties.group('First digit')
proportions = by_digit.column('count')/num_counties
by_digit = by_digit.with_columns(
    'Observed proportion', proportions,
    'Benford predicted proportion', benford_model
)
by_digit.drop('count').barh('First digit')

Null hypothesis: ____

Alternative hypothesis: ____

Test statistic: ___

Fill in the blank with "Bigger" or "Smaller":

___ values of the test statistic favor the alternative

In [ ]:
observed_tvd = sum(abs(proportions - benford_model))/2
observed_tvd

In [ ]:
sample_proportions(num_counties, benford_model)

In [ ]:
simulated_frequencies = sample_proportions(num_counties, benford_model)
tvd_from_sim = sum(abs(simulated_frequencies - benford_model))/2
tvd_from_sim

In [ ]:
def simulate_county_first_digits():
    simulated_frequencies = sample_proportions(num_counties, benford_model)
    tvd_from_sim = sum(abs(simulated_frequencies - benford_model))/2
    return tvd_from_sim

In [ ]:
simulated_tvds = make_array()

for i in np.arange(10000):
    simulated_tvds = np.append(simulated_tvds, simulate_county_first_digits())

In [ ]:
Table().with_column("TVD Predicted by Benford's Law", simulated_tvds).hist(0)

In [ ]:
np.count_nonzero(simulated_tvds >= observed_tvd) / 10000

Are the data consistent with the null hypothesis?

### Percentiles

In [ ]:
# Manually compute the 55th percentile.
x = make_array(43, 20, 51, 7, 28, 34)

In [ ]:
# Step 1. Sort the data
np.sort(x)

In [ ]:
# Step 2. Figure out where 55th percentile would be.

In [ ]:
# OR: 1 Line of Code
percentile(55, x)

In [ ]:
# If we tried to compute which element to take...
55 / 100 * 6

### Sample Median

In [ ]:
chi = Table.read_table('chicago_salary_2021.csv')
chi = chi.where("Full or Part-Time", "F").where("Annual Salary", are.above(0)).drop("Full or Part-Time","Hourly Rate", "Typical Hours")

In [ ]:
# Who is making the most money
chi.sort('Annual Salary', descending=True).show(5)

In [ ]:
# Who is making the least money
chi.sort('Annual Salary', descending=False).show(5)

In [ ]:
pop_median = percentile(50, chi.column('Annual Salary'))
pop_median

In [ ]:
our_sample = chi.sample(100, with_replacement=False)
our_sample.show(5)

In [ ]:
percentile(50, our_sample.column('Annual Salary'))

In [ ]:
chi_bins = np.arange(0, 300000, 15000)
chi.hist('Annual Salary', bins=chi_bins)
plots.title('Population Distribution');

In [ ]:
our_sample.hist('Annual Salary', bins=chi_bins)
plots.title('Sample Distribution');

# Variability of the Estimate

In [ ]:
def generate_sample_median(samp_size):
    our_sample = chi.sample(samp_size, with_replacement=False)
    return percentile(50, our_sample.column('Annual Salary'))

In [ ]:
sample_median = generate_sample_median(100)
sample_median

In [ ]:
error = sample_median - pop_median
error

# Quantifying Uncertainty

In [ ]:
sample_medians = make_array()

for i in np.arange(1000):
    new_median = generate_sample_median(100)
    sample_medians = np.append(sample_medians, new_median)

In [ ]:
med_bins = np.arange(80000, 100000, 1750)
Table().with_column(
    'Sample Medians', sample_medians
).hist(bins = med_bins)

plots.scatter(pop_median, -1e-6, color="red");

In [ ]:
err_bins = np.arange(-6000, 6000, 1700)
Table().with_column(
    'Errors', sample_medians - pop_median
).hist(bins = err_bins)

plots.scatter(0, -1e-6, color="red");

# Bootstrap

In [ ]:
# Take a bootstrap (re)sample of size 300, WITH replacement
boot_sample = our_sample.sample(100, with_replacement=True)
boot_sample.hist('Annual Salary', bins=chi_bins)
plots.title('Bootstrap sample');

print("Population Median =       ", pop_median)
print("Our Sample Median =       ", sample_median)
print("Bootstrap Sample Median = ", 
      percentile(50,boot_sample.column('Annual Salary')))

In [ ]:
def one_bootstrap_median():
    boot_resample = our_sample.sample()
    return percentile(50, boot_resample.column('Annual Salary'))

In [ ]:
bootstrap_medians = make_array()
for i in np.arange(1000):
    new_median = one_bootstrap_median()
    bootstrap_medians = np.append(bootstrap_medians, new_median)

In [ ]:
Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=med_bins)

plots.scatter(pop_median, -1e-6, color="red");
plots.scatter(sample_median, -1e-6, color="blue");

## Confidence Intervals

The confidence interval is an interval based on the middle 95% of bootstrap samples.  The interval will be shown in yellow, the sample median (our estimate) in blue, and the true population median (the parameter) in red.

In [ ]:
left = percentile(2.5, bootstrap_medians)
right = percentile(97.5, bootstrap_medians)

Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=med_bins)

plots.plot([left, right], [-1e-6,-1e-6], color="gold",lw=3, zorder=1);
plots.scatter(pop_median, -1e-6, color="red", zorder=2);
plots.scatter(sample_median, -1e-6, color="blue", zorder=2);